# VeL-Ui Framework #

## Introduction ##
This tutorial will guide you through setting up and interacting with the VelUI, a user interface built on top of the LogView framework. VelUI allows you to visually build and evaluate queries on event logs, providing a dynamic and intuitive way to explore process mining data.


## Loading the VelUI ##

First, ensure you have the necessary dependencies installed :

- dash==2.0.0
- jupyter-dash==0.4.0
- dash-bootstrap-components
- dash feffery-antd-components



## Setting Up the LogView Library

To use the LogView library in this project, we need to ensure that Python can locate it. If the library is not installed as a package but is instead a local module in a specific directory, we can add that directory to the Python path manually.


In [1]:
import sys
import os

# Add the path to your module to Logview Library
path_to_your_module = os.path.dirname(os.getcwd())
sys.path.insert(0, path_to_your_module)

## Read an event log file ##
For this tutorial, we use the Road Traffic Fine Management (RTFM) event log, which records data about the process of managing and handling road traffic fines by a local police force in Italy. The RTFM log contains 561,470 events grouped into 150,370 cases recorded between January 2000 and June 2013, including 11 different activities and 12 data attributes.

More details about the event log can be found [here](https://link.springer.com/article/10.1007/s00607-015-0441-1#Sec8).\
The event log used in this tutorial is already provided in the dataset folder.\
The raw event log files can also be downloaded [here](https://data.4tu.nl/articles/dataset/Road_Traffic_Fine_Management_Process/12683249?file=24018146).



In [2]:
from dash import jupyter_dash
from dash import Dash, html, dcc, callback, Output, Input
from Vel import Vel
import constants


vel_instance = Vel(logName="Road_Traffic_Fine_Management_Process")

In [3]:
vel_instance.run_Query_Builder()

Running Query Builder
App is ready to run at : http://127.0.0.1:8051


Selected log: None
Labels: []
Tag values: []
Selected log: None
Labels: []
Tag values: []
Labels:Selected log: None
 []
Tag values: []


In [4]:
vel_instance.log_view.get_summary()

+----+--------------------+-------------+----------------+--------------+
|    | source_log         | query       | result_set     | labels       |
|----+--------------------+-------------+----------------+--------------|
|  0 | initial_source_log | amount_zero | rs_amount_zero | [['unpaid']] |
+----+--------------------+-------------+----------------+--------------+
+----+-------------+-----------------+
|    | query       | predicates      |
|----+-------------+-----------------|
|  0 | amount_zero | (amount <= 0.0) |
+----+-------------+-----------------+


{'evaluations':            source_log        query      result_set      labels
 0  initial_source_log  amount_zero  rs_amount_zero  [[unpaid]],
 'queries':          query       predicates
 0  amount_zero  (amount <= 0.0)}

The Vel class is initialized with the path to a log file, automatically reads it, and prepares it for analysis. The initialization also sets up default column names for the case ID, timestamp, and activity columns.

Using `Changing Default Column Names()`:

The changeDefaultNames method allows you to manually define the specific columns in your dataframe that correspond to the case ID, timestamp, and activity. This is essential for ensuring the LogView framework correctly interprets your event log data.

In [3]:
vel_instance.changeDefaultNames("New Case ID", "New Timestamp", "New Activity")
vel_instance.CASE_ID_COL

'New Case ID'

Using `setLog()`:

The getLog method allows you to map your event log columns (Case ID, Timestamp, and Activity) and initialize the LogView directly through a web-based UI.

- Column Selection: Choose the appropriate columns directly from the table.
- Role Assignment: Assign selected columns to the Case ID, Timestamp, and Activity roles interactively.

In [3]:
from dash import jupyter_dash
from dash import Dash, html, dcc, callback, Output, Input
from Vel import Vel
import constants


app = vel_instance.setLog()

app.run_server(mode='inline', port=constants.SETLOGPORT)

### Instantiating a LogView object ###

The LogViewBuilder class allows instantiating a LogView object, which serves as the central interface for creating LogView instances. It provides a single point of access for interacting with the different framework components.

`initLogView()`:
The initLogView method simplifies the process of initializing a LogView object from raw event log data

In [7]:
vel_instance.initLogView()

## Evaluate Queries ##

In **LogView**, a _query_ is a combination (conjunction) of one or more predicates, which are similar to case filters. 
Predicates allow you to retrieve cases from the log that meet the conditions specified by the predicate. For example, a predicate such as 'amount ≤ 100' sets a condition on the values of the 'amount' attribute within the case.

### Query Predicates ###
The following predicates are currently supported in LogView.

**Filter on attribute values:**
- `EqToConstant`: Keeps cases that contain at least an event with the given attribute equal to a constant value.
- `NotEqToConstant`: Keeps cases that do not contain any event with the given attribute equal to a constant value.
- `GreaterEqualToConstant`: Keeps cases that contain at least an event with the given attribute greater than or equal to a constant value.
- `GreaterThanConstant`: Keeps cases that contain at least an event with the given attribute greater than a constant value.
- `LessEqualToConstant`: Keeps cases that contain at least an event with the given attribute lower than or equal to a constant value.
- `LessThanConstant`: Keeps cases that contain at least an event with the given attribute lower than a constant value.
- `StartWith`: Keeps cases starting with the specified activities.
- `EndWith`: Keeps cases ending with a given activity.
- `DurationWithin`: Keeps cases with durations within a specified range in seconds.
- `SumAggregate`: Sums the values of the specified attribute, grouping by the specified columns.
- `MaxAggregate`: Finds the maximum value of the specified attribute, grouping by the specified columns.
- `MinAggregate`: Finds the minimum value of the specified attribute, grouping by the specified columns.


(!WIP)`Query_Builder` creates a web application using Dash that serves as a visual query builder. Users can add queries, select predicates, configure conditions, and execute queries—all within an intuitive UI. The method handles everything from setting up the layout to managing callbacks that update the UI based on user interactions.

One of the core features of Query_Builder is its ability to dynamically manage queries


- Multiple Query Tabs: Users can create, manage, and switch between multiple queries using dynamically generated tabs.

- Dynamic Input Fields: Input fields adjust dynamically based on the selected predicate, allowing users to provide the necessary parameters

- Updating Conditions: The method provides detailed UI elements for configuring conditions within each query, such as selecting attributes, setting values, and defining predicates.

- Query Preview: The UI displays a real-time preview of the query string as users configure conditions and select predicates

- Query Execution and Results Display: Execute queries directly from the UI, and the results are displayed in an interactive DataTable

- Visual Feedback

